In [2]:
import json
import pprint
import csv

1) Необходимо прочитать файл cinema.json, используя библиотеку json. https://docs.python.org/3/library/json.html

Создать новые файлы cinema2.json и cinema2.csv, в которых для каждого кинотеатра будут отображены только следующие поля из исходного файла: 

* ID, 
* Краткое наименование (CommonName), 
* Адрес (текстовой строкой, только улица-дом, без индекса и района), 
* Сайт, 
* Рабочие часы по дням недели (в формате словаря "День недели: часы работы"), 
* Долгота, 
* Широта.

In [3]:
cinema_list = []

In [4]:
with open('cinema.json','rt') as file:
    data = json.load(file)

In [5]:
def working_hours(dicti):
    dictionary = {}
    for i in dicti:
        dictionary[i['DayWeek']] = i['WorkHours']
    return dictionary

In [6]:
def restruct_dicti(dicti):
    restruct = []
    for row in dicti:
        dictionary = {}
        dictionary.update(row)
        dictionary.update(row['WorkingHours'])
        del dictionary['WorkingHours']
        restruct.append(dictionary)
    return restruct

In [7]:
for row in data:
    cinema_list.append({
        "ID": row['CommonName'],
        "CommonName": row['global_id'],
        "Address": row['ObjectAddress'][0]['Address'],
        "WebSite": row['WebSite'],
        'WorkingHours': working_hours(row['WorkingHours']),
        'Longitude': row['geoData']['coordinates'][0][0],
        'Latitude': row['geoData']['coordinates'][0][0],
    })

In [8]:
with open('cinema2.json', 'wt', encoding='utf-8') as file:
    json.dump(cinema_list, file, ensure_ascii=False)

In [9]:
with open('cinema2.csv', 'w', newline='') as csv_file:
    dicti = restruct_dicti(cinema_list)
    headers = list(dicti[0].keys())
    csv_writer = csv.DictWriter(csv_file, delimiter=';', fieldnames = headers)
    csv_writer.writeheader()
    for row in dicti:
        csv_writer.writerow(row)

2) Присутствуют два csv-файла. В одном ID и координаты узлов графа (points.csv), в другом - координаты ребер графа, полученные с географической карты (polylines.csv). Необходимо к координатам из ребер графа привязать соответствующие им узлы графа (смотреть по близости их координат) и создать csv файл с полями "ID начального узла ребра" и "ID конечного узла ребра". 

**Важно**: координаты узлов могут минимально расходиться с координатами начала и концов ребер, поэтому свести по точному соответствию не получится.

Справка по работе с csv: https://docs.python.org/3/library/csv.html

In [10]:
lst3 = []

with open('polylines.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=';')
    for row in csv_reader:
        if row != []:
            lst3.append(row[0].replace('"', '').replace('),(', ');(').split(";"))
            
with open('polylines_remastered.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    
    for i in lst3:
        writer.writerow(i)
        
with open('polylines_remastered.csv', 'r') as f:
    csv_reader = csv.reader(f, delimiter=';')
    for row in csv_reader:
        if row != []:
            lst3.append(row[0].replace('"', '').replace('),(', ');(').split(";"))
            
with open('polylines_remastered.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    
    for i in lst3:
        writer.writerow(i)        
            

In [11]:
points = []

In [12]:
with open('points.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    current_id = 1
    for row in csv_reader:
        if len(row) > 0:
            points.append({
                'id': current_id,
                'lat': float(row[0]),
                'lon': float(row[1])
            })
            current_id += 1

In [46]:
print(points)

[{'id': 1, 'lat': 43.93784, 'lon': 56.28922}, {'id': 2, 'lat': 44.000857, 'lon': 56.263747}, {'id': 3, 'lat': 43.945512, 'lon': 56.280012}, {'id': 4, 'lat': 44.017494, 'lon': 56.316729}, {'id': 5, 'lat': 44.013892, 'lon': 56.315785}, {'id': 6, 'lat': 44.027977, 'lon': 56.31892}, {'id': 7, 'lat': 44.027914, 'lon': 56.318251}, {'id': 8, 'lat': 43.874796, 'lon': 56.32532}, {'id': 9, 'lat': 44.052061, 'lon': 56.293911}, {'id': 10, 'lat': 44.0473, 'lon': 56.294106}, {'id': 11, 'lat': 44.001234, 'lon': 56.322594}, {'id': 12, 'lat': 44.025561, 'lon': 56.309344}, {'id': 13, 'lat': 44.011188, 'lon': 56.321246}, {'id': 14, 'lat': 44.013847, 'lon': 56.320338}, {'id': 15, 'lat': 43.826413, 'lon': 56.362237}, {'id': 16, 'lat': 44.066425, 'lon': 56.299056}, {'id': 17, 'lat': 44.066775, 'lon': 56.284469}, {'id': 18, 'lat': 43.783231, 'lon': 56.380009}, {'id': 19, 'lat': 44.007262, 'lon': 56.317467}, {'id': 20, 'lat': 43.965185, 'lon': 56.23438}, {'id': 21, 'lat': 43.978489, 'lon': 56.295095}, {'id': 

In [29]:
lst_full = []
with open('polylines_remastered.csv', 'r', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=';')
    counter = 0
    for row in reader:
        lst = []
        for el in row:
            lst.append(point_remaster(el, points))
        print(counter, row)
        counter += 1
        lst_full.append([i for i in lst if i != None ])
print(lst_full)

0 ['(44.1652,56.1896)', '(44.04,56.2379)', '(44.0108,56.2366)', '(44.0053,56.2371)', '(44.0024,56.239)', '(44.0034,56.241)', '(44.0006,56.2399)', '(43.9941,56.2404)', '(43.9909,56.2407)', '(43.9818,56.2409)', '(43.9687,56.244)', '(43.9692,56.2455)', '(43.9735,56.2474)', '(43.9778,56.2455)', '(43.9779,56.2472)', '(43.9828,56.2465)', '(43.9854,56.2442)', '(43.9854,56.2452)', '(43.9864,56.2459)', '(43.9915,56.2443)']
1 ['(43.9865,56.2459)', '(43.9847,56.2491)']
2 ['(44.0054,56.2372)', '(44.0002,56.2338)']
3 ['(43.9734,56.2474)', '(43.9677,56.2493)', '(43.9685,56.2541)', '(43.9732,56.2596)']
4 ['(43.9821,56.2409)', '(43.9797,56.2373)', '(43.9837,56.2357)', '(43.9746,56.2355)', '(43.9752,56.2317)', '(43.9651,56.2344)', '(43.9656,56.235)', '(43.9601,56.2385)', '(43.955,56.2439)']
5 ['(43.9651,56.2344)', '(43.9571,56.2315)', '(43.9452,56.2325)', '(43.9396,56.2314)']
6 ['(43.9571,56.2316)', '(43.9512,56.2257)']
7 ['(43.9505,56.2257)', '(43.96,56.2158)', '(43.9716,56.2152)', '(44.0158,56.2114)'

In [23]:
def point_remaster(point, dicti):
    point_tuple = (
       float(point.split(',')[0][1:]),
       float(point.split(',')[1][:-1])
    )
    for row in dicti:
        if -0.0005 <= point_tuple[0] - row['lat'] <= 0.0005 and -0.0005 <= point_tuple[1] - row['lon'] <= 0.0005:
            return row['id']

In [15]:
final_list = []

In [26]:
current_id = 1
for i, e in enumerate(lst_full, 1):
    for ind, el in enumerate(e[:-1]):
        final_list.append({
            "id": current_id,
            "graph_id": i,
            "point_from": el,
            "point to": e[ind+1]            
    })
    current_id += 1
print(final_list)   

[{'id': 1, 'graph_id': 1, 'point_from': 86, 'point to': 184}, {'id': 1, 'graph_id': 1, 'point_from': 184, 'point to': 434}, {'id': 1, 'graph_id': 1, 'point_from': 434, 'point to': 198}, {'id': 1, 'graph_id': 1, 'point_from': 198, 'point to': 199}, {'id': 1, 'graph_id': 1, 'point_from': 199, 'point to': 200}, {'id': 1, 'graph_id': 1, 'point_from': 200, 'point to': 197}, {'id': 1, 'graph_id': 1, 'point_from': 197, 'point to': 196}, {'id': 1, 'graph_id': 1, 'point_from': 196, 'point to': 195}, {'id': 1, 'graph_id': 1, 'point_from': 195, 'point to': 194}, {'id': 1, 'graph_id': 1, 'point_from': 194, 'point to': 23}, {'id': 1, 'graph_id': 1, 'point_from': 23, 'point to': 298}, {'id': 1, 'graph_id': 1, 'point_from': 298, 'point to': 52}, {'id': 1, 'graph_id': 1, 'point_from': 52, 'point to': 540}, {'id': 1, 'graph_id': 1, 'point_from': 540, 'point to': 245}, {'id': 1, 'graph_id': 1, 'point_from': 245, 'point to': 174}, {'id': 1, 'graph_id': 1, 'point_from': 174, 'point to': 155}, {'id': 1, 'g

In [17]:
with open("final/points.csv", 'w', newline="") as csv_file:
    headers = list(points[0].keys())
    csv_writer = csv.DictWriter(csv_file, delimiter=';', fieldnames = headers)
    csv_writer.writeheader()
    for i in points:
        csv_writer.writerow(i)

In [21]:
with open("final/graphs.csv", 'w', newline="") as csv_file:
    headers = list(final_list[0].keys())
    csv_writer = csv.DictWriter(csv_file, delimiter=';', fieldnames = headers)
    csv_writer.writeheader()
    for i in final_list:
        csv_writer.writerow(i)

PermissionError: [Errno 13] Permission denied: 'final/graphs.csv'

3) Создать json-файл, состоящий из ребер графа. Каждое ребро должно иметь уникальный ID (например, номер по порядку), к которому должны быть привязаны данные о начальном узле ("Source"), данные о конечном узле ребра ("Target"). Каждый узел в json должен содержать исходный id, long (долготу) и lat (широту).

In [49]:

def inf_point(point_id):
    dicti_return = {}
    for i in points:
            if i['id'] == point_id:
                dicti_return['id'] = i['id']
                dicti_return['long'] = i['lon']
                dicti_return['lat'] = i['lat']
    return(dicti_return)


In [69]:
lst4=[]
counter = 1
for d in final_list:
    lst4.append(f'Graph_Id: {d["graph_id"]}, Rib_ID: {counter}, Source: {inf_point(d["point_from"])}, Target: {inf_point(d["point to"])}')
    counter += 1
print(lst4)

["Graph_Id: 1, Rib_ID: 1, Source: {'id': 86, 'long': 56.189729, 'lat': 44.165527}, Target: {'id': 184, 'long': 56.237637, 'lat': 44.040248}", "Graph_Id: 1, Rib_ID: 2, Source: {'id': 184, 'long': 56.237637, 'lat': 44.040248}, Target: {'id': 434, 'long': 56.236531, 'lat': 44.010828}", "Graph_Id: 1, Rib_ID: 3, Source: {'id': 434, 'long': 56.236531, 'lat': 44.010828}, Target: {'id': 198, 'long': 56.237131, 'lat': 44.005367}", "Graph_Id: 1, Rib_ID: 4, Source: {'id': 198, 'long': 56.237131, 'lat': 44.005367}, Target: {'id': 199, 'long': 56.238932, 'lat': 44.002528}", "Graph_Id: 1, Rib_ID: 5, Source: {'id': 199, 'long': 56.238932, 'lat': 44.002528}, Target: {'id': 200, 'long': 56.240868, 'lat': 44.003453}", "Graph_Id: 1, Rib_ID: 6, Source: {'id': 200, 'long': 56.240868, 'lat': 44.003453}, Target: {'id': 197, 'long': 56.239868, 'lat': 44.00048}", "Graph_Id: 1, Rib_ID: 7, Source: {'id': 197, 'long': 56.239868, 'lat': 44.00048}, Target: {'id': 196, 'long': 56.240303, 'lat': 43.994102}", "Graph_I

In [70]:
with open('final/TG.json', 'w') as f:
    json.dump(lst4, f)